# Standby consumption analysis

OpenGrid has a standby consumption analysis. This analysis will extract the minimum consumption during a specified period of the day. By default, the minimum over the entire day is found, but the method can be tuned to analyse eg. only the night. 

### Imports and loading data

In [ ]:
import opengrid as og
import pandas as pd
plt = og.plot_style()

For a correct standby power analysis **we need high-resolution data, eg. minute-values**.  With hourly values we would often overestimate the real standby power as fridges, freezers or other intermittent short loads are included in the hourly consumption measure.  

In this demo we use electricity consumption data with 1-minute resolution for a single sensor. First, we load the dataframe and make a quick plot to show the data.

In [ ]:
ts = og.datasets.get('elec_power_min_1sensor')
ax = ts.plot()
fig = ax.set_ylabel('W')

### Compute standby power

The daily standby power consumption is the daily minimum electric power. However, we can also compute the standby power for each 12h period, as shown below.  Both results are compared in the graph. 

In [ ]:
standby = og.analysis.standby(ts) # returns a pandas Series

In [ ]:
fig = plt.plot(standby.index, standby.values, marker='D', linestyle='-.')
plt.title('Daily standby power')
_ = fig[0].axes.set_ylabel('W')

In [ ]:
print("The mean daily standby power is {:.1f} W".format(standby.mean()))
print("The minimum daily standby power is {:.1f} W".format(standby.min()))

### Interpretation

From the results we can see that the daily standby power is lower than 60 W on all days except the 8th of October. This is a low value, most dwellings will have a higher standby power. The increased standbly power on the 8th of October is caused by **equipment (typically lighting) is not turned off as excpected**. 

We can now compute the share of the standby consumption in the total electricity consumption over this period. We consider the minimum of 51.1 W as the *real standby power* and all consumption above as *variable load*. 

In [ ]:
p_tot = ts.sum()/60/1000 # conversion to kWh
p_standby = standby.min() * len(ts) / 60/1000 # kWh
p_var = p_tot - p_standby

print("Standby power: {:.1%} of total consumption.".format(p_standby/p_tot))

In [ ]:
fig1, ax1 = plt.subplots(figsize=(5,5))
ax1.pie([p_standby, p_var], 
        explode=(0.1, 0), 
        labels=['Standby consumption', 'Variable consumption'], 
        autopct='%1.1f%%',
        shadow=True, 
        startangle=90)
_ = ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.